In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import numpy as np
import xarray as xr

import doralite
import gfdl_utils.core as gu
import CM4Xutils

import xbudget
import regionate
import xwmb
import warnings

In [14]:
def time_mean(ds, ds_weights=None):
    coords = ds.coords
    if ds_weights is None:
        weights = xr.DataArray(
            ds.time_bounds.diff("time_bounds").values.astype("float64"),
            coords={"time":ds.time},
            dims=("time",)
        )
    else:
        weights = xr.DataArray(
            ds_weights.time_bounds.diff("time_bounds").values.astype("float64"),
            coords={"time":ds_weights.time},
            dims=("time",)
        )
    ds_mean = ds.weighted(weights).mean("time")
    ds_mean = ds_mean.assign_coords({c:coord for (c,coord) in coords.items() if c not in ds_mean.coords})
    return ds_mean

In [38]:
models = ["CM4Xp25", "CM4Xp125"]
toSv = 1e-6/1035.

wmb_vars = [
    'boundary_fluxes',
    'diffusion',
    'spurious_numerical_mixing',
    'mass_source',
    'mass_tendency',
    'convergent_mass_transport_along'
]

for model in models:
    wmt_west = time_mean(
        xr.open_zarr(f"../../data/wmb_{model}_Labrador_2010-2024.zarr").sel(exp="forced")
    ).drop_vars(["exp", "lat", "lon", "wet"])*toSv
    wmt_east = time_mean(
        xr.open_zarr(f"../../data/wmb_{model}_IrmingerIceland_2010-2024.zarr").sel(exp="forced")
    ).drop_vars(["exp", "lat", "lon", "wet"])*toSv
    wmt_west = wmt_west.sel(xh=slice(-80, 10), yh=slice(45, 75))
    wmt_west = wmt_west[wmb_vars].assign_coords(wmt_west.coords)
    
    wmt_east = wmt_east.sel(xh=slice(-80, 10), yh=slice(45, 75))
    wmt_east = wmt_east[wmb_vars].assign_coords(wmt_east.coords)
    
    sec_ranges = {
        "west": {
            "Davis Strait Inflow": np.arange(110, wmt_west.lon_sect.size),
            "OSNAP-W": np.arange(0, 110)
        },
        "east": {
            "Nordic Sea Overflows": np.arange(0, 224),
            "OSNAP-E": np.arange(224, wmt_east.lon_sect.size)
        }
    }

    wmt_west["convergent_mass_transport_Davis"] = wmt_west["convergent_mass_transport_along"].isel(sect=sec_ranges["west"]["Davis Strait Inflow"]).sum("sect")
    wmt_west["convergent_mass_transport_OSNAP-W"] = wmt_west["convergent_mass_transport_along"].isel(sect=sec_ranges["west"]["OSNAP-W"]).sum("sect")

    wmt_east["convergent_mass_transport_Nordic"] = wmt_east["convergent_mass_transport_along"].isel(sect=sec_ranges["east"]["Nordic Sea Overflows"]).sum("sect")
    wmt_east["convergent_mass_transport_OSNAP-E"] = wmt_east["convergent_mass_transport_along"].isel(sect=sec_ranges["east"]["OSNAP-E"]).sum("sect")

    wmt_west.to_netcdf(f"../../data/plotting_ready/wmb_OSNAP-west_{model}.nc", mode="w")
    wmt_east.to_netcdf(f"../../data/plotting_ready/wmb_OSNAP-west_{model}.nc", mode="w")

In [41]:
# Surface water mass transformation maps in CM4Xp125
wmt_map_west = xr.concat([
    xr.open_zarr(f"../../data/wmb_boundary_flux_maps_CM4Xp125_Labrador_2010-2014.zarr").sel(exp="forced"),
    xr.open_zarr(f"../../data/wmb_boundary_flux_maps_CM4Xp125_Labrador_2015-2019.zarr").sel(exp="forced"),
    xr.open_zarr(f"../../data/wmb_boundary_flux_maps_CM4Xp125_Labrador_2020-2024.zarr").sel(exp="forced")
], dim="time").mean("time")
wmt_map_east = xr.concat([
    xr.open_zarr(f"../../data/wmb_boundary_flux_maps_CM4Xp125_IrmingerIceland_2010-2014.zarr").sel(exp="forced"),
    xr.open_zarr(f"../../data/wmb_boundary_flux_maps_CM4Xp125_IrmingerIceland_2015-2019.zarr").sel(exp="forced"),
    xr.open_zarr(f"../../data/wmb_boundary_flux_maps_CM4Xp125_IrmingerIceland_2020-2024.zarr").sel(exp="forced")
], dim="time").mean("time")

swmt = (wmt_map_west['boundary_fluxes'] + wmt_map_east['boundary_fluxes'])/wmt_map_west['areacello']*toSv*1e11
swmt = swmt.where(swmt!=0).rename("swmt")
swmt.to_netcdf("../../data/plotting_ready/swmt_OSNAP_CM4Xp125.nc", mode="w")